# EXERCISE: Glass Identification (aka "Glassification")

## TASK 1: read the data into a DataFrame

In [1]:
import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data'
col_names = ['id','ri','na','mg','al','si','k','ca','ba','fe','glass_type']
df = pd.read_csv(url, names=col_names, index_col='id')

## TASK 2: briefly explore the data

In [10]:
%matplotlib notebook

## TASK 3: convert into binary classification problem 

Convert this into a binary classification problem. Create a new DataFrame column called "binary":
1.	If type of glass = 1/2/3/4, set binary = 0.
2.	If type of glass = 5/6/7, set binary = 1.

## TASK 4: create a feature matrix (X) using all features 

## TASK 5: create a response vector (y)

## TASK 6: split X and y into training and testing set

## TASK 7: fit a KNN model on the training set using K=5

## TASK 8: make predictions on the testing set and calculate testing accuracy

## TASK 9: write a for loop that computes testing accuracy for a range of K values

## TASK 10: plot K value versus testing accuracy to choose on optimal value for K

In [ ]:
%matplotlib inline


## TASK 11: calculate the null accuracy (see instructions for explanation)

In [8]:
df.binary.value_counts()

370 µs ± 5.78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## TASK 12: search for useful features

In [ ]:
# redo exercise using only those features
feature_cols = ['mg','ba']

In [9]:
df.corr()

,ri,na,mg,al,si,k,ca,ba,fe,glass_type,binary
ri,1.000000,-0.191885,-0.122274,-0.407326,-0.542052,-0.289833,0.810403,-0.000386,0.143010,-0.164237,-0.134327
na,-0.191885,1.000000,-0.273732,0.156794,-0.069809,-0.266087,-0.275442,0.326603,-0.241346,0.502898,0.452336
mg,-0.122274,-0.273732,1.000000,-0.481799,-0.165927,0.005396,-0.443750,-0.492262,0.083060,-0.744993,-0.758289
al,-0.407326,0.156794,-0.481799,1.000000,-0.005524,0.325958,-0.259592,0.479404,-0.074402,0.598829,0.585928
si,-0.542052,-0.069809,-0.165927,-0.005524,1.000000,-0.193331,-0.208732,-0.102151,-0.094201,0.151565,0.148072
k,-0.289833,-0.266087,0.005396,0.325958,-0.193331,1.000000,-0.317836,-0.042618,-0.007719,-0.010054,0.053774
ca,0.810403,-0.275442,-0.443750,-0.259592,-0.208732,-0.317836,1.000000,-0.112841,0.124968,0.000952,0.040670
ba,-0.000386,0.326603,-0.492262,0.479404,-0.102151,-0.042618,-0.112841,1.000000,-0.058692,0.575161,0.523403
fe,0.143010,-0.241346,0.083060,-0.074402,-0.094201,-0.007719,0.124968,-0.058692,1.000000,-0.188278,-0.194903
glass_type,-0.164237,0.502898,-0.744993,0.598829,0.151565,-0.010054,0.000952,0.575161,-0.188278,1.000000,0.941680
